In [1]:
import pandas as pd
import numpy as np

In [2]:
trip = pd.read_csv('data/tripP')
train = pd.read_csv('data/trip_train.csv', usecols=['id'])
test = pd.read_csv('data/trip_test.csv', usecols=['id'])

/home/pabloernesto/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train = train.merge(trip, on='id', how='left')
test = test.merge(trip, on='id', how='left')

In [4]:
train.head(1)

,id,duration,date,start_station_id,subscription_type,time,year,month,day,dayofweek,date.1,holiday
0,907649,396,2015-08-27,50,1,516,2015,8,27,4,NaN,NaN


In [11]:
from sklearn import pipeline
from sklearn.svm import SVR
#from sklearn.kernel_approximation import Nystroem
#fmap_rbf = Nystroem(kernel='rbf')
#fmap_lin = Nystroem(kernel='linear')
#fmap_pl2 = Nystroem(kernel='poly', degree=2)
#fmap_pl3 = Nystroem(kernel='poly', degree=3)
#fmaps = [fmap_rbf, fmap_lin, fmap_pl2, fmap_pl3]

In [18]:
from sklearn import preprocessing
cols = ['start_station_id', 'subscription_type', 'time', 'year', 'month', 'day','dayofweek']
X = train[cols].apply(lambda x: preprocessing.MinMaxScaler().fit_transform(x), axis=0)
y = train.duration

/home/pabloernesto/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)
/home/pabloernesto/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/pabloernesto/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/h

In [19]:
X.sample(5)

,start_station_id,subscription_type,time,year,month,day,dayofweek
373154,0.646341,1.0,0.305073,0.5,0.636364,0.100000,0.000000
545081,0.487805,1.0,0.372481,1.0,0.181818,0.766667,0.166667
291169,0.817073,1.0,0.785962,1.0,0.636364,0.766667,0.000000
282654,0.451220,1.0,0.743572,1.0,0.545455,0.700000,0.333333
450357,0.634146,1.0,0.792217,1.0,0.090909,0.300000,0.166667


In [26]:
regressor = SVR(max_iter=1000, kernel='sigmoid')
#pipeline.Pipeline([('feature_map', fmaps[0]),
#                  ('svr', regressor)])
regressor.fit(X, y)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='sigmoid', max_iter=1000, shrinking=True, tol=0.001,
  verbose=False)

In [22]:
X_test = test[cols].apply(lambda x: preprocessing.MinMaxScaler().fit_transform(x), axis=0)
y_test = test.duration

/home/pabloernesto/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/pabloernesto/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/pabloernesto/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.re

In [27]:
from sklearn.metrics import mean_squared_error
mean_squared_error(regressor.predict(test[cols]), test.duration)

471452665.36948228

In [32]:
pd.Series(regressor.predict(test[cols])).describe()

count    119998.000000
mean      21787.116423
std           0.009885
min       21786.762258
25%       21787.114872
50%       21787.115129
75%       21787.116202
max       21787.757009
dtype: float64